In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import random
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
import itertools
import pickle
import operator
import copy
import os
%matplotlib notebook

# Generate synthetic networks  $\bigcirc --- \bigcirc$
We genrate two types of networks (i) Erdos-Renyi and (ii) Scale-free.

__ER Network__

In [ ]:
# parameters
N = int(1e4) # network size
k = 3.5 # average degree - from Makse & Morone 2015
seeds = random.sample(range(1000),100)

save_path = 'some_folde_somewhere'

Generate 100 ER graphs and save them for later reuse

In [ ]:
for ii in range(0,100):
    seed = seeds[ii]

    G = nx.fast_gnp_random_graph(N, k/(N-1.0), directed = False,seed = seed)
    GCC = max(nx.connected_component_subgraphs(G),key=len)
    
    print ii,seed,len(GCC)
    
    fileOut = open('%s/ER%03d.csv' % (save_path,ii),'w')
    for i,j in GCC.edges_iter():
        fileOut.write('%d,%d\n' % (i,j))
    fileOut.close()

__SF Networks__

Because SF networks generated by the configuration model ($\gamma = 3$) are very sparse ($N_{edges} \simeq N_{nodes} $) we have chosen to focus on networks generated using the exponent $\gamma = 2.5$

_Methodology_
1. first pick N numbers from a Pareto distribution with slope $\gamma$
2. construct degree sequence, by rounding to integer values in interval [$k_{min}$,$k_{max}$]
3. check if degree sequence is valid (possible to construct a network from it)
4. construct graph using the configuration model
5. check if graph is ok, i.e. no multi-edges, and self-loops
6. If graph only has multiedges, then perform double edge swaps until multi graph is destroyed
7. save giant connected component to file

In [ ]:
#N = int(1e4) #+ 1000
N = 1000 #+ 1000
gamma = 2.5
k_min = 1
k_max = 50 # approx 0.5% of network with N = 1e4

save_path = 'some_path_somewhere_'

In [ ]:
n_ok = 0
ii = 0
while n_ok < 1:
    # pick sample from distribution
    sf_sample = [random.paretovariate(gamma-1) for i in range(N)]

    # construct degree sequence by rounding to integers and enforce k_min & k_max thresholds
    deg_seq=[min(k_max, max( int(round(s)),k_min )) for s in sf_sample] # enforce k_min AND k_max
    #deg_seq=[max( int(round(s)),k_min ) for s in sf_sample] # enforce k_min

    # check if sequence is valid
    if nx.is_valid_degree_sequence_havel_hakimi(deg_seq):
    #if nx.is_valid_degree_sequence(deg_seq):
        
        # pair nodes using configuration model
        G = nx.configuration_model(deg_seq)
        ii += 1
        
        # check if graph is ok
        #if G.selfloop_edges() == []:
        if G.selfloop_edges() == [] or len(list(G.selfloop_edges())) <=3:
            
            # remove (often very few) self loops
            G.remove_edges_from(G.selfloop_edges())
            
            # perform double edgeswaps
            H = nx.double_edge_swap(G, nswap=10*N, max_tries=100*N)
            
            if len(H.edges()) == len(set(H.edges())):
                # transform from multigraph to undirected graph
                H = nx.Graph(H)
                # only select giant connected component
                GCC = max(nx.connected_component_subgraphs(H),key=len)

                print n_ok, len(GCC.nodes()),len(GCC.edges()), gamma, min(deg_seq), max(deg_seq)

                # output network
                fileOut = open('%s/SF1000_gamma%0.1f.csv' % (save_path,gamma),'w')
                #for i,j in GCC.edges_iter():
                for i,j in GCC.edges():
                    fileOut.write('%d,%d\n' % (i,j))
                fileOut.close()

                # increment counter
                n_ok += 1